# Background

Great work! The desk now has the price data they need. The final ingredient before they can begin trading with the client is the pricing model. Alex tells you the client wants to start trading as soon as possible. They believe the winter will be colder than expected, so they want to buy gas now to store and sell in winter in order to take advantage of the resulting increase in gas prices. They ask you to write a script that they can use to price the contract. Once the desk are happy, you will work with engineering, risk, and model validation to incorporate this model into production code.

The concept is simple: any trade agreement is as valuable as the price you can sell minus the price at which you are able to buy. Any cost incurred as part of executing this agreement is also deducted from the overall value. 

So, for example, if I can purchase a million MMBtu of natural gas in summer at $2/MMBtu, store this for four months, and ensure that I can sell the same quantity at $3/MMBtu without incurring any additional costs, the value of this contract would be ($3-$2) *1e6 = $1million. If there are costs involved, such as having to pay the storage facility owner a fixed fee of $100K a month, then the 'value' of the contract, from my perspective, would drop by the overall rental amount to $600K. Another cost could be the injection/withdrawal cost, like having to pay the storage facility owner $10K per 1 million MMBtu for injection/withdrawal, then the price will further go down by $10K to $590K. Additionally, if I am supposed to foot a bill of $50K each time for transporting the gas to and from the facility, the cost of this contract would fall by another $100K. Think of the valuation as a fair estimate at which both the trading desk and the client would be happy to enter into the contract.




# Task 2

You need to create a prototype pricing model that can go through further validation and testing before being put into production. Eventually, this model may be the basis for fully automated quoting to clients, but for now, the desk will use it with manual oversight to explore options with the client. 

You should write a function that is able to use the data you created previously to price the contract. ***The client may want to choose multiple dates to inject and withdraw a set amount of gas***, so your approach should generalize the explanation from before. Consider all the cash flows involved in the product.

The input parameters that should be taken into account for pricing are:

- Injection dates : date X amount of gas bought at $p 
- Withdrawal dates : date y amount of gas sold at $k
- The prices at which the commodity can be purchased/sold on those dates.
- The rate at which the gas can be injected/withdrawn.
- The maximum volume that can be stored.
- Storage costs.


Write a function that takes these inputs and gives back the value of the contract. You can assume there is no transport delay and that interest rates are zero. Market holidays, weekends, and bank holidays need not be accounted for. Test your code by selecting a few sample inputs.

In [1]:
# Importing modules

import pandas as pd
import numpy as np

## visualization 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore') ## ignoring warning


In [29]:
## let's work with volumes
def get_contract_value_dates(injection_dates : list[str], withdrawl_dates: list[str], max_volume: float, injection_withdrawl_vol:float):
    injection_ = pd.DataFrame({'dates': pd.to_datetime(injection_dates), 'action': [injection_withdrawl_vol]*len(injection_dates) })
    withdrawl_ = pd.DataFrame({'dates': pd.to_datetime(withdrawl_dates), 'action': [-injection_withdrawl_vol]*len(withdrawl_dates)})
    gas_transactions = pd.concat([injection_, withdrawl_]).sort_values('dates')
    gas_transactions['current_vol_gas'] = np.cumsum(gas_transactions.action)
    
    ## current gas cannot be greater than max and cannot be negative!
    neg_gas = gas_transactions[gas_transactions.current_vol_gas < 0].index.values.tolist()
    excess_gas = gas_transactions[gas_transactions.current_vol_gas > max_volume].index.values.tolist()
    stop_list = neg_gas + excess_gas
    stop_list.sort()
    return stop_list
def get_contract_value(injection_dates : list[str], injection_prices: list[float], withdrawl_dates: list[str], withdrawl_prices: list[float], max_volume: float,injection_withdrawl_cost:float, injection_withdrawl_vol:float, storage_cost_variable_pm:float = 0, storage_cost_fixed_pm :float = 0)->dict:
    ''' 
        injection_dates : list[str], ascending dates only!
        injection_prices: list[float], 
        withdrawl_dates: list[str], ascending dates only!!
        withdrawl_prices: list[float],
        max_volume: float 
        injection_withdrawl_cost:float, 
        injection_withdrawl_vol:float,
        storage_cost_pm:float

        Note: everything is per MMBtu

        ASSUMPTIONS: at each injection date x MMBtu are injected and on each withdrawl x is withdrawed.
                     Months are calculated as continous periods of 30 days. If stored for 45 days, the storage cost would be 1.5 times the monthly cost
                     That is storage costs are incurred as per consumption
        
    '''
    assert(len(injection_dates) == len(injection_prices))
    assert(len(withdrawl_dates) == len(withdrawl_prices))

    if(len(withdrawl_dates) - len(injection_dates) != 0):
        if(len(withdrawl_dates) - len(injection_dates) < 0):
            print('')
            raise Exception("more injection than withdrawl")
        if(len(withdrawl_dates) - len(injection_dates) > 0):
            raise Exception("more withdrawl than injection")
    
    ## checking if excess vol is injected or withdrawn
    if len(get_contract_value_dates(injection_dates = injection_dates, withdrawl_dates= withdrawl_dates, max_volume= max_volume, injection_withdrawl_vol= injection_withdrawl_vol)) != 0:
       raise Exception("volume limits breached mid contract")

        
    
    expected_sale_cf = injection_withdrawl_vol*(np.array(withdrawl_prices) - np.array(injection_prices))
    
    ## calculate months (30 day periods between insertion and extraction)
    months =  np.around((pd.to_datetime(withdrawl_dates) - pd.to_datetime(injection_dates)).days / 30, decimals=0)
    
    # calculate expected_injection_withdrawl_costs
    expected_injection_withdrawl_costs = injection_withdrawl_vol*injection_withdrawl_cost

    # calculate_expected_storage_costs

    expected_storage_costs = months* (injection_withdrawl_vol * storage_cost_variable_pm)

    value = np.sum( expected_sale_cf - expected_injection_withdrawl_costs - expected_storage_costs) - storage_cost_fixed_pm* np.around((pd.to_datetime(withdrawl_dates).max() - pd.to_datetime(injection_dates).min()).days / 30)

    results_dict = {'value' : value,'expected_sale_cf': expected_sale_cf, 'months':months,
                     'expected_injection_withdrawl_costs':  expected_injection_withdrawl_costs, 'expected_storage_costs': expected_storage_costs}
    
    return results_dict

    

In [30]:
get_contract_value(injection_dates = ['19/06/2023', '19/07/2023'], injection_prices=[2, 3], withdrawl_dates= ['19/06/2023','19/10/2023'], withdrawl_prices= [3,5], max_volume= 500000000,injection_withdrawl_cost = 0.01, injection_withdrawl_vol= 1000000, storage_cost_variable_pm = 0, storage_cost_fixed_pm = 100000)

{'value': 2580000.0,
 'expected_sale_cf': array([1000000, 2000000]),
 'months': Index([0.0, 3.0], dtype='float64'),
 'expected_injection_withdrawl_costs': 10000.0,
 'expected_storage_costs': Index([0.0, 0.0], dtype='float64')}

## testing

In [19]:
# case 1: no variable storage costs, only fixed per month storage costs.
injection_dates = ['19/01/2023', '19/02/2023', '19/05/2023']
withdrawl_dates = ['19/04/2023', '19/07/2023', '19/09/2023']
injectiontion_withdrawl_vol = 1000000
injectiontion_cost = 0.01 # per MMBtu; effective cost per injection/withdrawl = $10000
storage_cost_fixed_pm = 100000
max_volume = 3000000 #3 mil
contract_details = get_contract_value(injection_dates = injection_dates, injection_prices=[2, 2,2], withdrawl_dates= withdrawl_dates, withdrawl_prices= [3,3,3], max_volume= 3000000,injection_withdrawl_cost = 0.01, injection_withdrawl_vol= injectiontion_withdrawl_vol,  storage_cost_fixed_pm = storage_cost_fixed_pm)
contract_details['value']

2170000.0

In [28]:
# case 1: no variable storage costs, only fixed per month storage costs.
injection_dates = ['19/01/2023', '19/02/2023', '19/05/2023']
withdrawl_dates = ['19/04/2023', '19/07/2023', '19/09/2023']
injectiontion_withdrawl_vol = 1000000
injectiontion_cost = 0.01 # per MMBtu; effective cost per injection/withdrawl = $10000
storage_cost_fixed_pm = 100000
max_volume = 3000000 #3 mil
contract_details = get_contract_value(injection_dates = injection_dates, injection_prices=[2, 2,2], withdrawl_dates= withdrawl_dates, withdrawl_prices= [3,3,3], max_volume= 3000000,injection_withdrawl_cost = 0.01, injection_withdrawl_vol= injectiontion_withdrawl_vol,  storage_cost_fixed_pm = storage_cost_fixed_pm, storage_cost_variable_pm= 0)
contract_details['value']

2170000.0

In [27]:
## variable cost
injection_dates = ['19/01/2023', '19/02/2023', '19/05/2023']
withdrawl_dates = ['19/04/2023', '19/07/2023', '19/09/2023']
injectiontion_withdrawl_vol = 1000000
injectiontion_cost = 0.01 # per MMBtu; effective cost per injection/withdrawl = $10000
storage_cost_variable_pm= 0.01 ## charge of 0.01 per MMBtu stored per month 
max_volume = 3000000 #3 mil
contract_details = get_contract_value(injection_dates = injection_dates, injection_prices=[2, 2,2], withdrawl_dates= withdrawl_dates, withdrawl_prices= [3,3,3], max_volume= 3000000,injection_withdrawl_cost = 0.01, injection_withdrawl_vol= injectiontion_withdrawl_vol,  storage_cost_fixed_pm = 0, storage_cost_variable_pm= storage_cost_variable_pm)
contract_details

{'value': 2850000.0,
 'expected_sale_cf': array([1000000, 1000000, 1000000]),
 'months': Index([3.0, 5.0, 4.0], dtype='float64'),
 'expected_injection_withdrawl_costs': 10000.0,
 'expected_storage_costs': Index([30000.0, 50000.0, 40000.0], dtype='float64')}